In [2]:
import re
import pandas as pd

In [35]:
# 파일 읽기
info_4 = pd.read_csv("place_info_4.csv")
kakao1 = pd.read_csv("/Users/jisookwon/Hackathon/Crawling/kakao_map/kakao_place_info_list.csv")
kakao2 = pd.read_csv("/Users/jisookwon/Hackathon/Crawling/kakao_map/kakao_place_info_list2.csv")
print(info_4.shape)
print(kakao1.shape)
print(kakao2.shape)

# 두 데이터프레임을 아래로 concat
kakao = pd.concat([kakao1, kakao2], ignore_index=True)
print(kakao.shape)

(4092, 10)
(2008, 7)
(186, 7)
(2194, 7)


In [14]:
# 결측값 제거
info_4_clean = info_4.dropna(subset=['business_hours'])
kakao_clean = kakao.dropna(subset=['business_hours'])
# info_4와 kakao의 business_hours에서 겹치는 부분 찾기
matching_rows = info_4_clean[info_4_clean['business_hours'].isin(kakao_clean['business_hours'])]
# 결과 출력
print(len(matching_rows))

36


In [18]:
# info_4와 kakao의 business_hours에서 겹치는 부분 찾기
matching_rows = info_4[info_4['business_hours'].isin(kakao['business_hours'])]
print(len(matching_rows))
# matching_rows에서 결측값이 몇 개인지 확인
missing_values_count = matching_rows['business_hours'].isna().sum()
print(f"결측값 개수: {missing_values_count}")
# matching_rows에서 결측값이 아닌 부분만 선택
non_missing_rows = matching_rows[matching_rows['business_hours'].notna()]
print(len(non_missing_rows))
# non_missing_rows를 CSV 파일로 저장
non_missing_rows.to_csv('place_kakao.csv', index=False)

683
결측값 개수: 647
36


In [25]:
df = pd.read_csv("place_kakao.csv")

In [45]:
# 영업시간 kakao

import re

def parse_business_hours1(hours_str):
    if not isinstance(hours_str, str):
        return None
    
    days = {
        "월": [None, None, None],
        "화": [None, None, None],
        "수": [None, None, None],
        "목": [None, None, None],
        "금": [None, None, None],
        "토": [None, None, None],
        "일": [None, None, None],
    }
    
    def expand_day_range(day_range):
        order = ["월", "화", "수", "목", "금", "토", "일"]
        day_range = day_range.strip()
        if day_range == "매일":
            return order.copy()
        elif "~" in day_range:
            start, end = day_range.split("~")
            start_idx = order.index(start.strip())
            end_idx = order.index(end.strip())
            return order[start_idx:end_idx+1]
        elif "," in day_range:
            return [d.strip() for d in day_range.split(",")]
        else:
            return [day_range.strip()]
    
    # 영업시간 및 휴게시간 파싱
    segments = re.findall(r'영업시간:\s*(.*?)(?=\s*(?:영업시간:|공휴일:|$))', hours_str, re.DOTALL)
    
    for seg in segments:
        seg = seg.strip()
        if '휴게시간' in seg:
            # 휴게시간 처리
            match = re.match(r'([월화수목금토일~,매일]+)\s+휴게시간\s+(\d{2}:\d{2})\s*~\s*(\d{2}:\d{2})', seg)
            if not match:
                continue
            day_range, bt_start, bt_end = match.groups()
            target_days = expand_day_range(day_range)
            for day in target_days:
                if days[day][0]:
                    biz_start, biz_end = days[day][0].split('~')
                    days[day][0] = f"{biz_start}~{bt_start}"
                    days[day][1] = f"{bt_start}~{bt_end}"
                    days[day][2] = f"{bt_end}~{biz_end}"
        else:
            # 영업시간 처리
            match = re.match(r'([월화수목금토일~,매일]+)\s+(\d{2}:\d{2})\s*~\s*(\d{2}:\d{2})', seg)
            if not match:
                continue
            day_range, start, end = match.groups()
            target_days = expand_day_range(day_range)
            for day in target_days:
                days[day][0] = f"{start}~{end}"
    
    return days

# 테스트 데이터
test_data = [
    "매일 09:00 - 17:00 (11월~2월) 마감 1시간 전까지 입장 매일 09:00 - 18:00 (3~5월, 9~10월) 마감 1시간 전까지 입장 매일 09:00 - 18:30 (6~8월) 마감 1시간 전까지 입장 화요일 휴무 (휴무일이 공휴일과 겹칠경우 다음날 휴무)",
    "월~토 09:30 ~ 19:30",
]

# 결과 출력
for idx, data in enumerate(test_data):
    parsed = parse_business_hours1(data)
    print(f"📌 테스트 {idx+1} 원본: {data}")
    print("📅 정제 결과:")
    for day, times in parsed.items():
        print(f"   {day}: {times}")
    print("\n" + "="*50 + "\n")

# df2 = df.copy()
# df2['parsed_hours'] = df2['business_hours'].apply(parse_business_hours)
# print(df2['parsed_hours'].to_string())

📌 테스트 1 원본: 매일 09:00 - 17:00 (11월~2월) 마감 1시간 전까지 입장 매일 09:00 - 18:00 (3~5월, 9~10월) 마감 1시간 전까지 입장 매일 09:00 - 18:30 (6~8월) 마감 1시간 전까지 입장 화요일 휴무 (휴무일이 공휴일과 겹칠경우 다음날 휴무)
📅 정제 결과:
   월: [None, None, None]
   화: [None, None, None]
   수: [None, None, None]
   목: [None, None, None]
   금: [None, None, None]
   토: [None, None, None]
   일: [None, None, None]


📌 테스트 2 원본: 월~토 09:30 ~ 19:30
📅 정제 결과:
   월: [None, None, None]
   화: [None, None, None]
   수: [None, None, None]
   목: [None, None, None]
   금: [None, None, None]
   토: [None, None, None]
   일: [None, None, None]




In [32]:
# "이동하기"라는 단어가 포함된 행을 찾기
matching_rows = info_4[info_4['business_hours'].str.contains('이동하기', na=False)]
print(len(matching_rows))
print(matching_rows)

10
        id           name main_category category  \
62      63  그 남자의 파스타 대학로           음식점   이탈리아음식   
256    257            다모아           음식점       한식   
617    618         마로니에카페            카페   카페,디저트   
687    688        왕짜장 1호점           음식점      중식당   
702    703           백제식당           음식점  육류,고기요리   
1180  1181            전주집           음식점       한식   
1218  1219  혼다라멘 동대문종합시장점           음식점    일본식라면   
1298  1299          서대문곰탕           음식점   곰탕,설렁탕   
2441  2442           고향맛집           음식점       한식   
3815  3816    가을단풍길(삼청동길)          체험관광     계절거리   

                                          address  \
62                              서울 종로구 대학로 151 1층   
256   서울 종로구 종로 183 (인의동) 지2층 SF2016(인의동 효성주얼리시티)   
617                                서울 종로구 이화장길 66   
687                               서울 종로구 종로 300-1   
702                               서울 종로구 종로26길 18   
1180                                서울 종로구 삼봉로 81   
1218                   서울 종로구 종로 266 동대문종합시장B동 6층   


In [37]:
# 이동하기

import re

def parse_business_hours2(hours_str):
    if not isinstance(hours_str, str):
        return None
    
    days = {
        "월": [None, None, None],
        "화": [None, None, None],
        "수": [None, None, None],
        "목": [None, None, None],
        "금": [None, None, None],
        "토": [None, None, None],
        "일": [None, None, None],
    }
    
    def expand_day_range(day_range):
        order = ["월", "화", "수", "목", "금", "토", "일"]
        day_range = day_range.strip()
        if day_range == "매일":
            return order.copy()
        elif "~" in day_range:
            start, end = day_range.split("~")
            start_idx = order.index(start.strip())
            end_idx = order.index(end.strip())
            return order[start_idx:end_idx+1]
        elif "," in day_range:
            return [d.strip() for d in day_range.split(",")]
        else:
            return [day_range.strip()]
    
    # 영업시간 및 휴게시간 파싱
    segments = re.findall(r'이동하기:\s*(.*?)(?=\s*(?:이동하기:|공휴일:|$))', hours_str, re.DOTALL)
    
    for seg in segments:
        seg = seg.strip()
        if '휴게시간' in seg:
            # 휴게시간 처리
            match = re.match(r'([월화수목금토일~,매일]+)\s+휴게시간\s+(\d{2}:\d{2})\s*~\s*(\d{2}:\d{2})', seg)
            if not match:
                continue
            day_range, bt_start, bt_end = match.groups()
            target_days = expand_day_range(day_range)
            for day in target_days:
                if days[day][0]:
                    biz_start, biz_end = days[day][0].split('~')
                    days[day][0] = f"{biz_start}~{bt_start}"
                    days[day][1] = f"{bt_start}~{bt_end}"
                    days[day][2] = f"{bt_end}~{biz_end}"
        else:
            # 영업시간 처리
            match = re.match(r'([월화수목금토일~,매일]+)\s+(\d{2}:\d{2})\s*~\s*(\d{2}:\d{2})', seg)
            if not match:
                continue
            day_range, start, end = match.groups()
            target_days = expand_day_range(day_range)
            for day in target_days:
                days[day][0] = f"{start}~{end}"
    
    return days

# # 테스트 데이터
# test_data = [
#     "영업중 수정 제안 이동하기: 매일 12:00 ~ 23:00",
#     "영업중 수정 제안 이동하기: 월~토 09:30 ~ 19:30",
# ]

# # 결과 출력
# for idx, data in enumerate(test_data):
#     parsed = parse_business_hours(data)
#     print(f"📌 테스트 {idx+1} 원본: {data}")
#     print("📅 정제 결과:")
#     for day, times in parsed.items():
#         print(f"   {day}: {times}")
#     print("\n" + "="*50 + "\n")

# df2 = df.copy()
# df2['parsed_hours'] = df2['business_hours'].apply(parse_business_hours)
# print(df2['parsed_hours'].to_string())

In [46]:
# 네이버

import pandas as pd
import re

def parse_business_hours3(hours_str):
    if pd.isna(hours_str) or not isinstance(hours_str, str):
        return None
    
    days = {"월": [None, None, None], "화": [None, None, None], "수": [None, None, None], "목": [None, None, None], 
            "금": [None, None, None], "토": [None, None, None], "일": [None, None, None]}
    
    # "매일"인 경우 처리
    daily_match = re.search(r'매일\n(\d{2}:\d{2}) - (\d{2}:\d{2})', hours_str)
    if daily_match:
        time_range = f"{daily_match.group(1)}~{daily_match.group(2)}"
        for day in days:
            days[day] = [time_range, None, None]
        return days  # "매일"이 있는 경우 바로 반환
    
    # 정기휴무 처리
    holiday_pattern = re.compile(r'(월|화|수|목|금|토|일)\s*정기휴무\s*\((.*?)\)')
    holiday_match = holiday_pattern.findall(hours_str)
    for day, holiday in holiday_match:
        days[day] = [f"정기휴무 ({holiday})", None, None]
        
    # "휴무" 처리
    holiday_pattern = re.compile(r'(월|화|수|목|금|토|일)\s*휴무')
    holiday_match = holiday_pattern.findall(hours_str)
    for day in holiday_match:
        days[day] = ["휴무", None, None]
    
    # 각 요일별 영업시간 처리
    for day in days.keys():
        if days[day][0] and ("정기휴무" in days[day][0] or "휴무" in days[day][0]):
            continue  # 정기휴무인 경우 건너뜀
        
        # 요일별 시간 추출
        pattern = rf'{day}\s*(.*?)\s*(?=\s*(?:[월화수목금토일]|$))'
        match = re.search(pattern, hours_str, re.DOTALL)
        
        if match:
            time_ranges = match.group(1).split('\n')
            open_hours, break_times = [], []
            
            for time in time_ranges:
                time = time.strip()
                if '브레이크타임' in time:
                    break_times = re.findall(r'(\d{2}:\d{2} - \d{2}:\d{2})', time)
                elif re.match(r'\d{2}:\d{2} - \d{2}:\d{2}', time):
                    open_hours.append(time)
            
            # 브레이크타임이 있는 경우 시간 분할
            if open_hours and break_times:
                start, end = open_hours[0].split(' - ')
                bt_start, bt_end = break_times[0].split(' - ')
                days[day] = [f"{start}~{bt_start}", f"{bt_start}~{bt_end}", f"{bt_end}~{end}"]
            elif open_hours:
                days[day] = [open_hours[0].replace(' - ', '~'), None, None]
    
    return days

# 테스트 데이터
test_data = [
    "매일 09:00 - 17:00 (11월~2월) 마감 1시간 전까지 입장 매일 09:00 - 18:00 (3~5월, 9~10월) 마감 1시간 전까지 입장 매일 09:00 - 18:30 (6~8월) 마감 1시간 전까지 입장 화요일 휴무 (휴무일이 공휴일과 겹칠경우 다음날 휴무)",
    "월~토 09:30 ~ 19:30",
]

# # 결과 출력
# for idx, data in enumerate(test_data):
#     parsed = parse_business_hours3(data)
#     print(f"📌 테스트 {idx+1} 원본: {data}")
#     print("📅 정제 결과:")
#     for day, times in parsed.items():
#         print(f"   {day}: {times}")
#     print("\n" + "="*50 + "\n")
    
# test2 = test.copy()
# test2['parsed_hours'] = test2['business_hours'].apply(parse_business_hours)
# print(test2['parsed_hours'].to_string())


📌 테스트 1 원본: 매일 09:00 - 17:00 (11월~2월) 마감 1시간 전까지 입장 매일 09:00 - 18:00 (3~5월, 9~10월) 마감 1시간 전까지 입장 매일 09:00 - 18:30 (6~8월) 마감 1시간 전까지 입장 화요일 휴무 (휴무일이 공휴일과 겹칠경우 다음날 휴무)
📅 정제 결과:
   월: [None, None, None]
   화: [None, None, None]
   수: [None, None, None]
   목: [None, None, None]
   금: [None, None, None]
   토: [None, None, None]
   일: ['휴무', None, None]


📌 테스트 2 원본: 월~토 09:30 ~ 19:30
📅 정제 결과:
   월: [None, None, None]
   화: [None, None, None]
   수: [None, None, None]
   목: [None, None, None]
   금: [None, None, None]
   토: [None, None, None]
   일: [None, None, None]




In [41]:
info_4 = pd.read_csv("place_info_4.csv")

def apply_parse_function(row):
    if pd.notna(row) and '영업시간' in row:
        return parse_business_hours1(row)
    elif pd.notna(row) and '이동하기' in row:
        return parse_business_hours2(row)
    else:
        return parse_business_hours3(row)

info_4['business_hours'] = info_4['business_hours'].apply(apply_parse_function)
# CSV 파일로 저장
info_4.to_csv('place_info_5.csv', index=False)

In [43]:
info_5 = pd.read_csv("place_info_5.csv")

# 1. 데이터 타입 확인 및 변환
import ast

# 딕셔너리로 저장된 문자열을 실제 딕셔너리로 변환
info_5['business_hours'] = info_5['business_hours'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# 2. 딕셔너리 변환 함수 적용
def transform_business_hours(business_hours):
    """딕셔너리 → 슬래시 구분 문자열"""
    if isinstance(business_hours, dict):
        return "/".join([f"'{k}': {v}" for k, v in business_hours.items()])
    return business_hours  # 딕셔너리 아닌 경우 원본 유지

info_5['business_hours'] = info_5['business_hours'].apply(transform_business_hours)

# 3. 문자열 치환 (정기휴무 → 휴무)
info_5['business_hours'] = info_5['business_hours'].str.replace(
    '정기휴무', 
    '휴무', 
    regex=False
)

# 4. CSV 저장
info_5.to_csv('place_info_6.csv', index=False)
